In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# import runs
runs_df = pd.read_csv('runs.csv')
runs_df.info()

In [22]:
# remove orphaned runs
grouped = runs_df.groupby(['session', 'run'])

# Filter groups that have at least two different 'central' values
filtered_groups = [group for name, group in grouped if group['central'].nunique() > 1]

# Concatenate the filtered groups back into a single DataFrame
runs_df = pd.concat(filtered_groups)

In [23]:
# Fill has_ix values
# runs_df['has_ix'] = runs_df.groupby(['session', 'run'])['has_ix'].transform(lambda x: x.ffill().bfill())

In [ ]:
# get number of samples per num_robots
runs_df.groupby(['num_robots', 'central'])['run'].count()

In [ ]:
runs_df.head()

In [26]:
# calculate per-run statistics
runs_df['timed_out_robots'] = runs_df['num_robots'] - runs_df['successful_robots'] - runs_df['aborted_robots']
runs_df['success_perc'] = runs_df['successful_robots'] / runs_df['num_robots'] * 100.0
runs_df['failure_perc'] = 100.0 - runs_df['success_perc']
runs_df['timeout_perc'] = runs_df['timed_out_robots'] / runs_df['num_robots'] * 100.0
runs_df['abortion_perc'] = runs_df['aborted_robots'] / runs_df['num_robots'] * 100.0
runs_df['collision_perc'] = runs_df['collided_robots'] / runs_df['num_robots'] * 100.0
runs_df['robot_collision_perc'] = runs_df['robot_collided_robots'] / runs_df['num_robots'] * 100.0
runs_df['static_collision_perc'] = runs_df['static_collided_robots'] / runs_df['num_robots'] * 100.0
runs_df['robot_collisions_per_robot'] = runs_df['robot_collisions'] / runs_df['num_robots']
runs_df['static_collisions_per_robot'] = runs_df['static_collisions'] / runs_df['num_robots']

In [ ]:
# calculate statistics per (num_robots, central)

runs_stats = runs_df.groupby(['num_robots', 'central']).agg({
    'run': 'count',
    'failure_perc': 'mean',
    'timeout_perc': 'mean',
    'abortion_perc': 'mean',
    'collision_perc': 'mean',
    'robot_collision_perc': 'mean',
    'static_collision_perc': 'mean',
    'robot_collisions_per_robot': 'mean',
    'static_collisions_per_robot': 'mean',
    'avg_nav_time': 'mean',
    'max_nav_time': 'max'
}).rename(columns={
    'run': 'num_runs',
    'failure_perc': 'avg_failure_perc',
    'timeout_perc': 'avg_timeout_perc',
    'abortion_perc': 'avg_abortion_perc',
    'collision_perc': 'avg_collision_perc',
    'robot_collision_perc': 'avg_robot_collision_perc',
    'static_collision_perc': 'avg_static_collision_perc',
    'robot_collisions_per_robot': 'avg_r2r_collisions',
    'static_collisions_per_robot': 'avg_r2o_collisions'
})

runs_stats

In [ ]:
sns.barplot(runs_stats, y='avg_failure_perc', x='num_robots', hue='central')
plt.ylabel('Mean failure rate (%)')
plt.xlabel('Number of robots')

The mean failure rate is calculated as the mean of failure rates across runs with the same `num_robots` and `central`. A robot is considered to fail if it does not reach its destination upon the timeout (which can be caused by either timeouts or Nav2 navigation abortions).

In [ ]:
sns.barplot(runs_stats, y='avg_timeout_perc', x='num_robots', hue='central')
plt.ylabel('Mean timeout rate (%)')
plt.xlabel('Number of robots')

In [ ]:
sns.barplot(runs_stats, y='avg_abortion_perc', x='num_robots', hue='central')
plt.ylabel('Mean Nav2 goal abortion rate (%)')
plt.xlabel('Number of robots')

In [ ]:
sns.barplot(runs_stats, y='avg_robot_collision_perc', x='num_robots', hue='central')
plt.ylabel('Mean robot versus robot collision rate (%)')
plt.xlabel('Number of robots')

In [ ]:
sns.barplot(runs_stats, y='avg_r2r_collisions', x='num_robots', hue='central')
plt.ylabel('Mean robot collision count per robot')
plt.xlabel('Number of robots')

In [ ]:
sns.barplot(runs_stats, y='avg_r2o_collisions', x='num_robots', hue='central')
plt.ylabel('Mean object collision count per robot')
plt.xlabel('Number of robots')

In [ ]:
sns.barplot(runs_stats, y='avg_nav_time', x='num_robots', hue='central')
plt.ylabel('Mean navigation time of successful robots (sec)')
plt.xlabel('Number of robots')

In [ ]:
sns.barplot(runs_stats, y='max_nav_time', x='num_robots', hue='central')
plt.ylabel('Maximum navigation time of successful robots (sec)')
plt.xlabel('Number of robots')

In [ ]:
no_timeout_runs = runs_df[runs_df['timed_out_robots'] == 0]
highest_failure_rate_runs = no_timeout_runs.loc[no_timeout_runs.groupby(['num_robots', 'central'])['failure_perc'].idxmax()].sort_values(by=['num_robots', 'central'])
highest_failure_rate_runs

In [ ]:
most_collided_runs = no_timeout_runs.loc[no_timeout_runs.groupby(['num_robots', 'central'])['robot_collisions'].idxmax()].sort_values(by=['num_robots', 'central'])
most_collided_runs

In [ ]:
# robot collision stats
runs_df.groupby(['num_robots', 'central'])[['robot_collisions', 'static_collisions']].agg(['max', 'mean', 'median'])